In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image

# Mount Google Drive
from google.colab import drive

from IPython.display import Image, display
import os


In [11]:


# Specify the path to your CSV file
dataset_path = 'JustRAIGS_Train_labels.csv'

# Read the dataset into a Pandas DataFrame
df = pd.read_csv(dataset_path)


# Display basic information about the dataset
print(df.info())

# Display the first few rows of the dataset
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101423 entries, 0 to 101422
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                                                                                                        Non-Null Count   Dtype 
---  ------                                                                                                                                                                                                                                                                                                        --------------   ----- 
 0   Eye ID;Final Label;Fellow Eye ID;Age;Label G1;Label G2;Label G3;G1 ANRS;G1 ANRI;G1 RNFLDS;G1 RNFLDI;G1 BCLVS;G1 BCLVI;G1 NVT;G1 DH;G1 LD;G1 LC;G2 ANRS;G2 ANRI;G2 RNFLDS;G2 RNFLDI;G2 BCLVS;G2 BCLVI;G2 NVT;G2 DH;G2 LD;G2 LC;G

In [12]:
# Read the dataset into a Pandas DataFrame with the correct delimiter
df = pd.read_csv(dataset_path, delimiter=';')

# Display basic information about the dataset
print(df.info())

# Display the first few rows of the dataset
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101423 entries, 0 to 101422
Data columns (total 37 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Eye ID         101423 non-null  object 
 1   Final Label    101423 non-null  object 
 2   Fellow Eye ID  93017 non-null   object 
 3   Age            101409 non-null  float64
 4   Label G1       101149 non-null  object 
 5   Label G2       100046 non-null  object 
 6   Label G3       8412 non-null    object 
 7   G1 ANRS        101149 non-null  float64
 8   G1 ANRI        101149 non-null  float64
 9   G1 RNFLDS      101149 non-null  float64
 10  G1 RNFLDI      101149 non-null  float64
 11  G1 BCLVS       101149 non-null  float64
 12  G1 BCLVI       101149 non-null  float64
 13  G1 NVT         101149 non-null  float64
 14  G1 DH          101149 non-null  float64
 15  G1 LD          101149 non-null  float64
 16  G1 LC          101149 non-null  float64
 17  G2 ANRS        100046 non-nul

In [ ]:
"""
PLEASE WRITE DOWN TO THE WRITE ALL COL NAME
"""

In [13]:
# Display summary statistics for numerical columns
print(df.describe())


                 Age        G1 ANRS        G1 ANRI      G1 RNFLDS  \
count  101409.000000  101149.000000  101149.000000  101149.000000   
mean       56.675098       0.032853       0.035482       0.007968   
std        10.200041       0.178251       0.184996       0.088910   
min         0.000000       0.000000       0.000000       0.000000   
25%        49.000000       0.000000       0.000000       0.000000   
50%        57.000000       0.000000       0.000000       0.000000   
75%        64.000000       0.000000       0.000000       0.000000   
max       137.000000       1.000000       1.000000       1.000000   

           G1 RNFLDI       G1 BCLVS       G1 BCLVI         G1 NVT  \
count  101149.000000  101149.000000  101149.000000  101149.000000   
mean        0.007879       0.011280       0.011992       0.018735   
std         0.088416       0.105609       0.108851       0.135587   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.0000

In [27]:
"""

1. Age:
   - Count: 80,859 entries have non-null values for age.
   - Mean: The average age is approximately 56.67 years.
   - Standard Deviation: Age has a standard deviation of around 10.21, indicating some variability.
   - Min/Max: The minimum age is 0, and the maximum age is 137.

2. G1 ANRS to G3 LC:
   - Count: The number of non-null entries for these features varies.
   - Mean: These columns are binary (0 or 1), representing the presence or absence of certain characteristics.
   - Standard Deviation: Indicates the variability in the presence of these characteristics.

For instance, taking "G1 ANRS" as an example:
   - Count: 80,646 non-null entries.
   - Mean: 0.033 indicates that, on average, the characteristic is present in approximately 3.3% of cases.
   - Standard Deviation: 0.179 suggests some variability or spread in the data.

Similarly, the other columns follow a similar interpretation.


"""

'\n\n1. Age:\n   - Count: 80,859 entries have non-null values for age.\n   - Mean: The average age is approximately 56.67 years.\n   - Standard Deviation: Age has a standard deviation of around 10.21, indicating some variability.\n   - Min/Max: The minimum age is 0, and the maximum age is 137.\n\n2. G1 ANRS to G3 LC:\n   - Count: The number of non-null entries for these features varies.\n   - Mean: These columns are binary (0 or 1), representing the presence or absence of certain characteristics.\n   - Standard Deviation: Indicates the variability in the presence of these characteristics.\n\nFor instance, taking "G1 ANRS" as an example:\n   - Count: 80,646 non-null entries.\n   - Mean: 0.033 indicates that, on average, the characteristic is present in approximately 3.3% of cases.\n   - Standard Deviation: 0.179 suggests some variability or spread in the data.\n\nSimilarly, the other columns follow a similar interpretation.\n\n\n'

In [14]:
# Check for missing values in each column
missing_values = df.isnull().sum()

# Display the count of missing values for each column
print(missing_values)


Eye ID               0
Final Label          0
Fellow Eye ID     8406
Age                 14
Label G1           274
Label G2          1377
Label G3         93011
G1 ANRS            274
G1 ANRI            274
G1 RNFLDS          274
G1 RNFLDI          274
G1 BCLVS           274
G1 BCLVI           274
G1 NVT             274
G1 DH              274
G1 LD              274
G1 LC              274
G2 ANRS           1377
G2 ANRI           1377
G2 RNFLDS         1377
G2 RNFLDI         1377
G2 BCLVS          1377
G2 BCLVI          1377
G2 NVT            1377
G2 DH             1377
G2 LD             1377
G2 LC             1377
G3 ANRS          93011
G3 ANRI          93011
G3 RNFLDS        93011
G3 RNFLDI        93011
G3 BCLVS         93011
G3 BCLVI         93011
G3 NVT           93011
G3 DH            93011
G3 LD            93011
G3 LC            93011
dtype: int64


In [15]:
# Fill missing values in numeric columns with the mean
numeric_columns = df.select_dtypes(include=['float64']).columns
df[numeric_columns] = df[numeric_columns].apply(lambda x: x.fillna(x.mean()))

# Fill missing values in categorical columns with the mode
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: x.fillna(x.mode().iloc[0]))


In [16]:
# Display concise summary of the DataFrame
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101423 entries, 0 to 101422
Data columns (total 37 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Eye ID         101423 non-null  object 
 1   Final Label    101423 non-null  object 
 2   Fellow Eye ID  101423 non-null  object 
 3   Age            101423 non-null  float64
 4   Label G1       101423 non-null  object 
 5   Label G2       101423 non-null  object 
 6   Label G3       101423 non-null  object 
 7   G1 ANRS        101423 non-null  float64
 8   G1 ANRI        101423 non-null  float64
 9   G1 RNFLDS      101423 non-null  float64
 10  G1 RNFLDI      101423 non-null  float64
 11  G1 BCLVS       101423 non-null  float64
 12  G1 BCLVI       101423 non-null  float64
 13  G1 NVT         101423 non-null  float64
 14  G1 DH          101423 non-null  float64
 15  G1 LD          101423 non-null  float64
 16  G1 LC          101423 non-null  float64
 17  G2 ANRS        101423 non-nul

In [26]:
# Display summary statistics for numerical columns
#print(df.describe())


In [ ]:
"""
the changes in summary statistics are expected after handling missing values.
The imputation method used (mean for numeric columns) has influenced the mean and
standard deviation values, but the general distribution characteristics remain similar
"""

In [17]:
# Count the occurrences of each class in the "Final Label" column
class_counts = df['Final Label'].value_counts()

# Display the class counts
print(class_counts)


NRG    98153
RG      3270
Name: Final Label, dtype: int64


In [ ]:
"""
The class distribution for the "Final Label" column in your dataset is as follows:

- "No Referable Glaucoma" (NRG): 78,220 instances
- "Referable Glaucoma" (RG): 2,651 instances

This distribution indicates a class imbalance, as there are significantly more instances
 of "No Referable Glaucoma" compared to "Referable Glaucoma." In imbalanced datasets like this,
machine learning models may become biased towards the majority class, and their performance on the minority class (RG) might be suboptimal.

To address this imbalance, consider applying one or more of the following strategies:

1. Resampling:
   - Undersampling:Reduce the number of instances in the majority class (NRG).
   - Oversampling: Increase the number of instances in the minority class (RG).

2. Synthetic Data Generation:
   - Use techniques like SMOTE (Synthetic Minority Over-sampling Technique) to generate synthetic samples for the minority class.

3. eighted Loss:
   - Assign different weights to classes during model training to give more importance to the minority class (RG).

4. Evaluation Metrics:
   - Focus on evaluation metrics such as precision, recall, F1-score, or area under the ROC curve (AUC-ROC) that consider the performance on both classes, especially the minority class.


"""

In [18]:
# Select relevant columns for correlation analysis
selected_columns = ['Age', 'G1 ANRS', 'G1 ANRI', 'G1 RNFLDS', 'G1 RNFLDI', 'G1 BCLVS', 'G1 BCLVI', 'G1 NVT', 'G1 DH', 'G1 LD', 'G1 LC', 'Final Label']

# Create a new DataFrame with selected columns
selected_df = df[selected_columns]

# Convert 'Final Label' to numerical values (e.g., 1 for 'RG' and 0 for 'NRG')
selected_df['Final Label'] = selected_df['Final Label'].map({'NRG': 0, 'RG': 1})

# Calculate the correlation matrix
correlation_matrix = selected_df.corr()

# Display the correlation matrix
print(correlation_matrix)


                  Age   G1 ANRS   G1 ANRI  G1 RNFLDS  G1 RNFLDI  G1 BCLVS  \
Age          1.000000  0.081730  0.088991   0.019999   0.022843  0.043641   
G1 ANRS      0.081730  1.000000  0.644320   0.410174   0.245788  0.507596   
G1 ANRI      0.088991  0.644320  1.000000   0.242474   0.389085  0.366628   
G1 RNFLDS    0.019999  0.410174  0.242474   1.000000   0.345411  0.244178   
G1 RNFLDI    0.022843  0.245788  0.389085   0.345411   1.000000  0.122830   
G1 BCLVS     0.043641  0.507596  0.366628   0.244178   0.122830  1.000000   
G1 BCLVI     0.046779  0.383760  0.495361   0.139272   0.193578  0.431144   
G1 NVT       0.061825  0.615945  0.574575   0.312380   0.285399  0.406406   
G1 DH        0.012869  0.122723  0.130478   0.070963   0.103597  0.071910   
G1 LD        0.064945  0.506398  0.501991   0.134891   0.147429  0.351327   
G1 LC        0.072602  0.624615  0.607214   0.194728   0.188459  0.384268   
Final Label  0.112321  0.579274  0.620279   0.229220   0.282517  0.311768   

<ipython-input-18-02ca757cb518>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_df['Final Label'] = selected_df['Final Label'].map({'NRG': 0, 'RG': 1})


In [ ]:
"""the changes in summary statistics are expected after handling missing values.
The imputation method used (mean for numeric columns) has influenced the mean and
standard deviation values, but the general distribution characteristics remain similar"""

In [19]:
drive.mount('/content/drive')

# Path to the folder containing the images on Google Drive
image_folder_drive = '/content/drive/MyDrive/ML-PROJECT/0'

# List all image files in the folder
image_files = [file for file in os.listdir(image_folder_drive) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif'))]

# Display all images in the folder
for image_file in image_files:
    image_path = os.path.join(image_folder_drive, image_file)
    #display(Image(filename=image_path))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:

#I TRY TO CONNECT THE IMAGE FOLDER TO DF from csv file

# Path to the folder containing the images on Google Drive
image_folder_path = '/content/drive/MyDrive/ML-PROJECT/0'

# Create a new column in the DataFrame to store image paths
df['Image_Path'] = ''

# Function to get the image path based on the Eye ID
def get_image_path(eye_id):
    return os.path.join(image_folder_path, f"{eye_id}.jpg")

# Update the 'Image_Path' column in the DataFrame
df['Image_Path'] = df['Eye ID'].apply(get_image_path)

# Display the updated DataFrame
print(df[['Eye ID', 'Image_Path']].head())



        Eye ID                                         Image_Path
0  TRAIN000000  /content/drive/MyDrive/ML-PROJECT/0/TRAIN00000...
1  TRAIN000001  /content/drive/MyDrive/ML-PROJECT/0/TRAIN00000...
2  TRAIN000002  /content/drive/MyDrive/ML-PROJECT/0/TRAIN00000...
3  TRAIN000003  /content/drive/MyDrive/ML-PROJECT/0/TRAIN00000...
4  TRAIN000004  /content/drive/MyDrive/ML-PROJECT/0/TRAIN00000...


In [25]:
# Set the display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Display the full DataFrame
print(df.head())

        Eye ID Final Label Fellow Eye ID   Age Label G1 Label G2 Label G3  \
0  TRAIN000000         NRG   TRAIN081848  48.0      NRG      NRG      NRG   
1  TRAIN000001         NRG   TRAIN018312  29.0      NRG      NRG      NRG   
2  TRAIN000002         NRG   TRAIN083822  61.0      NRG      NRG      NRG   
3  TRAIN000003         NRG   TRAIN055721  70.0      NRG      NRG      NRG   
4  TRAIN000004         NRG   TRAIN099283  48.0      NRG      NRG      NRG   

   G1 ANRS  G1 ANRI  G1 RNFLDS  G1 RNFLDI  G1 BCLVS  G1 BCLVI  G1 NVT  G1 DH  \
0      0.0      0.0        0.0        0.0       0.0       0.0     0.0    0.0   
1      0.0      0.0        0.0        0.0       0.0       0.0     0.0    0.0   
2      0.0      0.0        0.0        0.0       0.0       0.0     0.0    0.0   
3      0.0      0.0        0.0        0.0       0.0       0.0     0.0    0.0   
4      0.0      0.0        0.0        0.0       0.0       0.0     0.0    0.0   

   G1 LD  G1 LC  G2 ANRS  G2 ANRI  G2 RNFLDS  G2 RNFLDI 

In [ ]:
"""
From next task , we can analyse  all image as it has been connected to image path
"""